In [1]:
import numpy as np
import shapely
import json
import matplotlib.pyplot as plt

import rasterio
import rasterio.plot as rplt
from rasterio.crs import CRS
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.io import MemoryFile
from rasterio.windows import Window as rWindow
from rasterio.enums import Resampling
from matplotlib_scalebar.scalebar import ScaleBar

from typing import List, Any
import os
import cv2
import pyproj
import pandas as pd
from sgl_processor_v02 import ProcessorSGL, ProcessorSGLOptions, Window, WindowCollection
from sgl_utils import ROI, ROICollection, Window, WindowCollection, RES, date_with_comma, get_angle
import geopandas as gpd

d:\OneDrive\02_Standards\[Code]\01_Projects\ASRP_Antarctic-Remote-Sensing-1\.venv_ASRP\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
optionsSGL = ProcessorSGLOptions()
optionsSGL.rangestart = "20191201"
optionsSGL.rangeend = "20200331"
processorSGL = ProcessorSGL(optionsSGL)
metas = processorSGL.get_metadata_from_csv(processorSGL.opts.output_folder, ["tile-181", "tile-182", "tile-183"])
coll = processorSGL.create_collection_from_meta("shackleton1920", metas)
rcoll = processorSGL.create_roicollection_from_csv("shackleton")

X=====================================================
PostProcessor started for 20191201 to 20200331
X=====================================================
=====> Get metadata from .csv files.
=====> Create collection from csv meta.
     | 69 windows created for collection.
     | 16 regions created.


# Just Images

count = 0

for date in coll.dates:
    ws = [w for w in coll.windows if w.metadata["date"] == date and w.name == "tile-181"]
    if len(ws) != 1:
        continue
    w = ws[0]
    tiff_rgb = rasterio.open(os.path.join(processorSGL.opts.input_folder, w.file_rgb))
    tiff = rasterio.open(os.path.join(processorSGL.opts.input_folder, w.file_gee))
    image = tiff_rgb.read()
    clouds = tiff.read(1)
    #clouds[clouds == 0] = np.nan
    lakes = tiff.read(2)
    #lakes[lakes == 0] = np.nan

    iceshelf = np.zeros(clouds.shape)
    iceshelf[clouds == 0] = 1

    fig, ax = plt.subplots(figsize = (12, 12))
    rplt.show(image, transform = tiff_rgb.transform, ax = ax)
    rplt.show(lakes, transform = tiff.transform, ax = ax, cmap = "Blues_r")
    rplt.show(clouds, transform = tiff.transform, ax = ax, cmap = cmap_clouds)
    plt.show()

    count += 1
    if count == 2:
        break

# Clipped Images

In [3]:
images = processorSGL.create_clipped_images(coll, max_images = 30, roi = rcoll.get("E"), satellites = ["S2", "L8"], dominant_satellite = "S2", print_metadata = True, show_bool = False, return_windows = False)

=====> Create Images
     | 2019-12-05 | L8
     | 2019-12-06 | S2
     | 2019-12-13 | L8
     | 2019-12-16 | S2
     | 2019-12-21 | L8
     | 2019-12-26 | S2
     | 2019-12-29 | L8
     | 2020-01-05 | S2
     | 2020-01-06 | L8
     | 2020-01-14 | L8
     | 2020-01-15 | S2
     | 2020-01-22 | L8
     | 2020-01-25 | S2
     | 2020-01-30 | L8
     | 2020-02-04 | S2
     | 2020-02-07 | L8
     | 2020-02-14 | S2
     | 2020-02-15 | L8
     | 2020-02-23 | L8
     | 2020-02-24 | S2
     | 2020-03-02 | L8
     | 2020-03-05 | S2
     | 2020-03-10 | L8
     | 2020-03-15 | S2


## Video from clipped Images

In [3]:
def create_videos_from_images(images, windows, videoname: str = "test", fps: float = 1.0, number_last_images = 3):
    img0 = images["rgb"][0]
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    video_path = os.path.join(processorSGL.opts.parent_output_folder, "videos", videoname + ".avi")
    video = cv2.VideoWriter(video_path, fourcc, float(fps), (img0.shape[2], img0.shape[1]))
    bool_cleaned = [rgb.max() > rgb.min() for rgb in images["rgb"]]
    rgb_cleaned = [img for (img, b) in zip(images["rgb"], bool_cleaned) if b]
    lakes_cleaned = [img for (img, b) in zip(images["lakes"], bool_cleaned) if b]
    clouds_cleaned = [img for (img, b) in zip(images["clouds"], bool_cleaned) if b]
    windows_cleaned = [w for (w, b) in zip(windows, bool_cleaned) if b]


    for frame_count, rgb in enumerate(rgb_cleaned):
        #converted_img = np.zeros((rgb.shape[1], rgb.shape[2], rgb.shape[0]))
        #converted_dmg = np.zeros(images["dmg"].shape)

        #dmg_converted = np.dstack([images["dmg"], images["dmg"], images["dmg"]])
        #dmg_converted[dmg_converted <= 0] = np.nan
        #print(images["dmg"].shape)
        #dmg_prepared = (images["dmg"][0] * 2 * 255).astype(np.uint8)

        #print(dmg_prepared.shape)
        #dmg_colour = cv2.applyColorMap(dmg_prepared, cv2.COLORMAP_HOT)
        #print(dmg_colour.shape)
        #dmg_mask = dmg_colour.copy()
        #dmg_mask[dmg_colour > 0] = 1
        #print(dmg_mask)
        #dmg_mask = dmg_mask.astype(bool)
        #rplt.show_hist(np.moveaxis(dmg_colour, 2, 0))



        for i in range(rgb.shape[0]):
            rgb[i, :, :] = (rgb[i, :, :] - np.min(rgb[i, :, :]))/(np.max(rgb[i, :, :]) - np.min(rgb[i, :, :]))
        rgb_converted = np.moveaxis(rgb, 0, 2)
        rgb_prepared = (rgb_converted * 255).astype(np.uint8)
        frame = cv2.cvtColor(rgb_prepared, cv2.COLOR_RGB2BGR)
        
        


        #alpha = 0.0
        #beta = (1.0 - alpha)
        #dst = cv2.addWeighted(frame, alpha, dmg_colour, beta, 0.0)
        #bg = frame.copy()
        #print(frame.shape, dmg_colour.shape)
        #bg[dmg_mask] = cv2.addWeighted(frame, alpha, dmg_colour, beta, 0.0)[dmg_mask]


        font = cv2.FONT_HERSHEY_SIMPLEX
    
        # Use putText() method for
        # inserting text on video
        cv2.putText(frame, 
                    windows_cleaned[frame_count].metadata["date"] + " | " + windows_cleaned[frame_count].satellite, 
                    (50, 100), 
                    font, 2, 
                    (0, 255, 255), 
                    4, 
                    cv2.LINE_4)
        #print(frame.shape)
        #print(images_cleaned[0].astype(np.uint8))
        video.write(frame)
        if frame_count == len(rgb_cleaned) - 1:
            for i in range(number_last_images):
                video.write(frame)

    video.release()

In [26]:
images, included_windows = processorSGL.create_clipped_images(coll, roi = rcoll.get(region), satellites = ["S2", "L8"], dominant_satellite = "S2", print_metadata = True, return_windows = True)
create_videos_from_images(images, included_windows, region, fps = 0.5)


     | 2019-12-05 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2019-12-06 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2019-12-13 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2019-12-16 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2019-12-21 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2019-12-26 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2019-12-29 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-01-05 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-01-06 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-01-14 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-01-15 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-01-22 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-01-30 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-02-04 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-02-07 | L8 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-02-14 | S2 | (3, 4496, 2369) | (1, 4496, 2369)
     | 2020-02-15 | L8 | (3, 4496, 2369) | (1, 4496, 236

In [4]:
for roi in rcoll.rois:
    print(roi.name)
    images, included_windows = processorSGL.create_clipped_images(coll, roi = rcoll.get(roi.name), satellites = ["S2", "L8"], dominant_satellite = "S2", show_bool = False, return_windows = True)
    create_videos_from_images(images, included_windows, roi.name, fps = 0.8)



A
=====> Create Images
B1
=====> Create Images
B
=====> Create Images
C
=====> Create Images
D
=====> Create Images
E1
=====> Create Images
E
=====> Create Images
F
=====> Create Images
G
=====> Create Images
H1
=====> Create Images
H
=====> Create Images
I
=====> Create Images
J
=====> Create Images
K
=====> Create Images
L
=====> Create Images
Z
=====> Create Images
